## Анализ текстовых данных

### Сбор данных

In [1]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек 
import time          # Библиотека для времени
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

Для сбора данных я выбрал новости футбола на сайте sports.ru. Отправная точка — https://www.sports.ru/football/topnews/

Доступно 20 страниц по 100 новостей.

Напишем функции: getPageLinks для парсинга очередной страницы со ссылками на новости, getNewsItemData для парсинга страницы с новостью.

In [2]:
def getPageLinks(page_number):
   
    # составляем ссылку на страницу поиска
    page_link = 'https://sports.ru/football/topnews/?page={}'.format(page_number)
    
    # запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return []
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    # наконец, ищем ссылки на новости и очищаем их от ненужных тэгов
    news_links = soup.findAll('a', attrs = {'class':'short-text'})
    news_links = [link.attrs['href'] for link in news_links]
    news_links = [link for link in news_links if link.find('/football/') == 0]
    
    return news_links

In [3]:
def getNewsItemData(page):
    
    # составляем ссылку на страницу новости
    page_link = 'https://sports.ru{}'.format(page)
    
    # запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки 
        return response.status_code
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # получаем текст новости
    content = soup.find('div', attrs={'class':'news-item__content'})
    content = content.get_text() if content else ""
    
    # получаем количество комментов
    comments = soup.find('a', attrs={'class':'news-item__comments-link'})
    comments = int(comments.get_text().split()[0]) if comments else 0
    
    return {'content': content, 'comments': comments}

Из новости мы получаем для дальнейшей работы текст и количество комментариев как целевую переменную. Создадим датафрейм для сохранения

In [4]:
df = pd.DataFrame(columns=['content','comments'])

In [5]:
from tqdm.notebook import tqdm

Теперь в цикле проходимся по 20 страницам со ссылками и во вложенном цикле заходим на страницы новостей.

In [7]:
for page_number in tqdm(range(1,21), desc='Pages'):
    # собрали хрефы с текущей страницы
    links = getPageLinks(page_number)  
    for link in tqdm(links, desc='News', leave=False):
        # иногда с первого раза страничка не парсится
        for i in range(5):
            try:
                # пытаемся собрать данные
                data = getNewsItemData(link)           
                # и закидываем их в таблицу
                df = df.append(data, ignore_index=True)  
                # если всё получилось - выходим из внутреннего цикла
                break
            except:
                # Иначе, пробуем еще несколько раз, пока не закончатся попытки
                print('Ошибка! Попробуем снова:', link)
                continue

Ошибка! Попробуем снова: /football/1095951796-inter-kalyari-onlajn-translyacziya-nachnetsya-v-1330.html


Ошибка! Попробуем снова: /football/1095579549-kuman-kategoricheski-protiv-perexoda-aguero-v-barsu.html


Ошибка! Попробуем снова: /football/1095128421-zlatan-o-vylete-ot-myu-milan-propustil-gol-kotoryj-ne-dolzhen-byl-prop.html


In [11]:
df.to_csv('sports.csv')

Данные сохранил в файл, дальше с ними работаем в новом ноутбуке.